<a name="about"></a>
About this notebook
======

This notebook assumes you have ran the local Iris notebook and you have not deleted the LOCAL_ROOT folder. In this notebook, we will use online prediction on a pre-trained Tensorflow model. This notebook will does not assume that the notebook "3. Iris Classification Cloud Training" was executed.

<a name="setup"></a>
Setting things up
=====

In [5]:
import datalab_structured_data as sd

In [6]:
import os
import pandas as pd

import tensorflow as tf
from tensorflow.python.lib.io import file_io

tf 1.0.0
sd 1.0.0


This notebook will write files during prediction. Please give a root folder you wish to use.

In [7]:
LOCAL_ROOT = './iris_notebook_workspace' # This should be the same as what was used in the local notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + '-iris-classification-datalab' # Feel free to change this line.

# No need to edit anything else in this cell.
LOCAL_TRAINING_DIR = os.path.join(LOCAL_ROOT, 'training')
CLOUD_TRAINING_DIR = os.path.join(CLOUD_ROOT, 'training')

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueError('LOCAL_ROOT not found. Did you run the local notebook?')
  
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-dev-iris-classification-datalab/...
ServiceException: 409 Bucket cloud-ml-dev-iris-classification-datalab already exists.


First, let us put the csv files on GCS and the output of training.

In [8]:
!gsutil -m cp -r {LOCAL_TRAINING_DIR} {CLOUD_TRAINING_DIR}

Copying file://./iris_notebook_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/transforms_file.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/evaluation_model/assets.extra/schema.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/model/assets.extra/schema.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/evaluation_model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/evaluation_model/assets.extra/transforms.json [Content-Type=application/json]...
Copying file://./

In [9]:
!gsutil ls {CLOUD_TRAINING_DIR}

gs://cloud-ml-dev-iris-classification-datalab/training/transforms_file.json
gs://cloud-ml-dev-iris-classification-datalab/training/evaluation_model/
gs://cloud-ml-dev-iris-classification-datalab/training/model/
gs://cloud-ml-dev-iris-classification-datalab/training/train/


Online Prediction using ML Engine
==========

We first have to deploy the model, and then we can make requests on it.

In [10]:
!gcloud beta ml models create iris_model --regions=us-central1
!gcloud beta ml versions create v1 --model iris_model --origin {CLOUD_ROOT}/training/model

Creating version (this might take a few minutes)......done.


In [11]:
sd.cloud_predict(
  model_name='iris_model',
  model_version='v1',
  data=['101,6.3,3.3,6,2.5', #virginica
        '100,5.7,2.8,4.1,1.3', #versicolor
        '13,4.8,3,1.4,0.1', #setosa
  ]
)

,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.940259,Iris-versicolor,0.0597013,Iris-setosa,3.96256e-05
1,100,Iris-versicolor,0.840116,Iris-virginica,0.145837,Iris-setosa,0.0140476
2,13,Iris-setosa,0.952213,Iris-versicolor,0.0477338,Iris-virginica,5.28481e-05


In [12]:
sd.cloud_predict(
  model_name='iris_model',
  model_version='v1',
  data=pd.DataFrame(
    [[101,6.3,3.3,6,2.5],
     [100,5.7,2.8,4.1,1.3],
     [13,4.8,3,1.4,0.1]])
)

,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.940259,Iris-versicolor,0.0597013,Iris-setosa,3.96256e-05
1,100,Iris-versicolor,0.840116,Iris-virginica,0.145837,Iris-setosa,0.0140476
2,13,Iris-setosa,0.952213,Iris-versicolor,0.0477338,Iris-virginica,5.28481e-05


Cleaning things up
=====

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [13]:
#!gsutil rm -fr {CLOUD_ROOT}

Removing gs://cloud-ml-dev-iris-classification-datalab/eval.csv#1488321127402905...
Removing gs://cloud-ml-dev-iris-classification-datalab/features.json#1488321128997494...
Removing gs://cloud-ml-dev-iris-classification-datalab/schema.json#1488321130576766...
Removing gs://cloud-ml-dev-iris-classification-datalab/train.csv#1488321125806455...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://cloud-ml-dev-iris-classification-datalab/cloud_preprocessing/numerical_analysis.json#1488320199532737...
Removing gs://cloud-ml-dev-iris-classification-datalab/cloud_preprocessing/preprocessing/numerical_analysis.json#1488321132233716...
Removing gs://cloud-ml-dev-iris-classification-datalab/cloud_preprocessing/preprocessing/schema.json#1488321132248018...
Removing gs://cloud-ml-dev-ir

To delete the models created, uncomment and run the next cell.

In [14]:
!gcloud beta ml versions delete v1 --model iris_model 
!gcloud beta ml models delete iris_model 

Deleting version [v1]......done.
Deleting model [iris_model]...done.
